# Provisioning

## Instances
Instances are sets of vms of various types that can be provisioned across the GreenLake for private cloud stack and a wide range of services like MySQL, Redis, ElasticSearch, PostgreSQL, Tomcat, nginx, Confluence, Jenkins, and more. There are a few important concept differentiators between what we call an instance and what amazon calls an instance. In GreenLake an instance can represent many vms that are of a set. For example. If you wanted to spin up a Mongo sharded replicaset, that requires 7 virtual machines. In GreenLake for private cloud we represent this as a singular instance with a specified layout and then represent all the associated services running within that virtual machine. The serverId represents the compute resource it was provisioned onto, (i.e. the virtual machine).


In this second lab, we will take care of some more advanced management features like snapshots or clones.

# Important Note
The HPE GreenLake Central API Acess token has a time to live duration of 15 minutes fro security reasons.

if during the course of the lab, you run an API call and get a result indicating your token is not valid : 


{
  "message": "Unable to read user authentication information.",
  "details": "Invalid access token provided: 'Unauthorized access: Token is not valid'",
  "recommendedActions": [
    "Obtain a valid token and resubmit the request."
  ],
  "errorCode": "UNAUTHORIZED"
}


Please go back to step4 to retrieve the GreenLake Central API Acess token to copy / paste it in the token.txt file.

Do not append content but replace existing token with the new one.

Please re run the next cell each time you update the token in the token.txt file

In [ ]:
##Generating GreenLake Service Client API Token
#the following token is only valid for 15 minutes , please re run the cell to renew the token

ACCESS_TOKEN=$(curl -X POST "https://iam.us1.greenlake-hpe.com/oauth2/buu0lcj6alfdq0ajli4g/ausanc3firBcvcYdT357/v1/token" --header 'Content-Type: application/x-www-form-urlencoded' \
--data-urlencode 'client_id={{ STUDENT_CLIENTID }}' \
--data-urlencode 'client_secret={{ STUDENT_CLIENTSECRET }}' \
--data-urlencode 'grant_type=client_credentials' \
--data-urlencode 'scope=hpe-tenant' |jq -r  .access_token) 

echo "your api token is: $ACCESS_TOKEN"

# VMaaS Variables
API_URL="https://client.greenlake.hpe.com/api/iac-vmaas/v1"
space="HPEDEV%20Workshop-on-Demand%20workshops"
location="HPE"

##GreenLake Central Login Details
echo "your operation context is:" "student{{ STDID }}"  "with Password:" "{{ PASSSTU }}{{ PASSSTU }}"

Let's start by checking how many instances are running here

In [ ]:
echo "Listing all available instances now"
curl -s -X GET \
  "$API_URL/instances?space=$space&location=$location" \
  -H "Authorization: $ACCESS_TOKEN" |jq -r

We will need the student group id as well, let 's get it as a variable now.


In [ ]:
echo "Getting your group Id now"

groupid=$(curl -s "$API_URL/groups?space=$space&location=$location&name=student{{ STDID }}" -H "Authorization: $ACCESS_TOKEN" |jq -r '.groups[].id')
echo "your student groupid is: $groupid"

As we covered creation / update / start / stop / restart / suspend in the frst lab.

Let's see snapshots and clones in details now.

Let's start by getting the the VM id to be used later. We'll make it a variable for ease of use.

In [ ]:
echo "Getting your VM Id now"

student{{ STDID }}vmid=$(curl -s --request GET "$API_URL/instances?space=$space&location=$location&max=25&name=vm-student{{ STDID }}-api" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instances[0].id')


echo "your vm id is "$student{{ STDID }}vmid

### Snapshot an instance 

[Create a Snaphot GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/SnapshotsInstance/)

- PUT /instances/:id/snapshot
-d '{snapshot}
--name VALUE Name
--description VALUE Description

## Caution !!  The snapshot takes a few minutes to complete. Please be patient...

In [ ]:
echo "Creating a snapshot for your student{{ STDID }} instance with id $student{{ STDID }}vmid"


curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid/snapshot?space=$space&location=$location" \
-H "Authorization:  $ACCESS_TOKEN" \
-H "Content-Type: application/json" \
-d '{ "snapshot": { "name": "api-vm-student{{ STDID }}-snap", "description": "A snapshot created via VMAAS API Call for student{{ STDID }} instance" }}' | jq -r

### List now your Instance Snapshot
- GET/api/instances/:id/snapshots


In [ ]:
echo "Listing your newly created snapshot"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid/snapshots?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r

### You can also clone or resize  an instance

[Clone an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/ClonesInstance/)

[Resize an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/ResizesInstance/)


- PUT /instances/:id/clone

- PUT /instances/:id/resize

Payload:

{
  "instance": {
    "id": 482,
    "plan": {
      "id": 419
    }
  },
  "volumes": [
    {
      "id": 643,
      "rootVolume": true,
      "name": "Hard disk 1",
      "size": 10,
      "sizeId": null,
      "storageType": 1,
      "datastoreId": "auto"
    }
  ],
  "deleteOriginalVolumes": true
}


### Let's Clone your existing VM now!

We will stop the vm before cloning it to prevent duplicate ips issues once the clone is up.

In [ ]:
echo "Cloning your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s -X PUT "$API_URL/instances/$student{{ STDID }}vmid/clone?space=$space&location=$location"\
  -H "Authorization: $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  --data-raw '{
    "name": "clone-vm-student{{ STDID }}-api",
    "group": {
      "id": '$groupid'
    }
  }'

### Retrieve the clone vm id now

Wait a minute or so for the clone to complete.

In [ ]:
echo "Getting your Clone VM Id now"

clonestudent{{ STDID }}vmid=$(curl -s --request GET "$API_URL/instances?space=$space&location=$location&max=25&name=clone-vm-student{{ STDID }}-api" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instances[0].id')

echo "your clonestudent{{ STDID }} vm id is "$clonestudent{{ STDID }}vmid


echo "Checking status of your clone-vm-student{{ STDID }}-api instance with id $clonestudent{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$clonestudent{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### What did we do so far ?  
Let's get the historical details for our original centos instance. We can see through this API  call the different steps we went through during the lab, from provisionning, snapshotting and finally cloning.

In [ ]:
curl -s -X GET "$API_URL/instances/$student{{ STDID }}vmid/history?space=$space&location=$location&force=on" \
-H "Authorization: $ACCESS_TOKEN" \
-H "Content-Type: application/json" |jq -r

### Deleting the original Instance here

As we don't need the instance for the rest of the workshop, we will now delete it. Besides, it will prevent Ip duplicate issues when dealing with the clone.

In [ ]:
##Getting original Instance id

echo "Getting your original VM Id now"

student{{ STDID }}vmid=$(curl -s --request GET "$API_URL/instances?space=$space&location=$location&max=25&name=vm-student{{ STDID }}-api" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instances[0].id')


echo "your vm id is "$student{{ STDID }}vmid

## Deleting  original instance
echo "Deleting your student{{ STDID }} instance now"

curl -s -X DELETE "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location&force=on" \
-H "Authorization: $ACCESS_TOKEN" \
-H "Content-Type: application/json" |jq -r


Note that we make use of the force option in the deletion API call. This option will make sure we delete the associated snapshots.

Clones will need to be deleted in a separate API call.

Let's check about the snapshots.

In [ ]:
echo "Listing your snapshot(s)"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid/snapshots?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r

### What about accessing the clone vm now ? 
Let's get the details for our clone centos instance. An IP address might help to connect. Some crededntials too.

In [ ]:
echo " Retrieving your student{{ STDID }} Clone VM IP Address now"

student{{ STDID }}clonevmip=$(curl -s --request GET "$API_URL/instances/$clonestudent{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instance.connectionInfo[0].ip')

echo "Your student{{ STDID }} clone instance IP Address is $student{{ STDID }}clonevmip "

#### Check if Clone vm is reachable 

In [ ]:
ping -c 5 $student{{ STDID }}clonevmip

####  Connect now to the clone vm

Click on the + button on the right hand side of the notebook UI to reach the  launcher page. Click then on the terminal tile in the other category at the bottom of the page.

This will open up a new tab allowing you a proper cli access to run the ssh commands.

user is `student` and password is `P@ssWkshp`


![SSH](Pictures/wod-greenlakeapi-ssh.PNG)

copy paste the content of the next cell in the terminal window to excute it

In [ ]:
ssh student@$student{{ STDID }}clonevmip

In [ ]:
echo "Listing your clone(s)"

curl -s -X GET "$API_URL/instances/$student{{ STDID }}vmid/?space=$space&location=$location"\
-H "Authorization: $ACCESS_TOKEN" \
-H "Content-Type: application/json" | jq -r

#### Delete now to the clone vm

In [ ]:
echo "Getting your Clone VM Id now"

clonestudent{{ STDID }}vmid=$(curl -s --request GET "$API_URL/instances?space=$space&location=$location&max=25&name=clone-vm-student{{ STDID }}-api" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instances[0].id')


echo "your clonestudent{{ STDID }} vm id is "$clonestudent{{ STDID }}vmid

## Deleting clone instance
echo "Deleting your student{{ STDID }} clone instance now"

curl -s -X DELETE "$API_URL/instances/$clonestudent{{ STDID }}vmid?space=$space&location=$location&force=on" \
-H "Authorization: $ACCESS_TOKEN" \
-H "Content-Type: application/json" |jq -r

## Summary:
In this lab, you managed to use the HPE GreenLake Private Cloud API to create your first snapshot and clone  before accessing the vm and deleting it.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#008000;"></i>&nbsp;&nbsp;Next Steps

Conclusion

</br>
 <a href="2-WKSHP-VMAAS-Instance.ipynb" target="New" title="Back: Authentification"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-Conclusion.ipynb" target="New" title="Next:Conclusion"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-VMAAS-Blueprint.ipynb#sc" target="New" title="Next: Tasks"><i class="fas fa-chevron-circle-right" style="color:#008000;"></i></a></h2>